# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [6]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

%%configure
{
    "--conf": "spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.sql.hive.convertMetastoreParquet=false",
    "--enable-glue-datacatalog" :"true",
    "--datalake-formats":"hudi"
}



Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
The following configurations have been updated: {'--conf': 'spark.serializer=org.apache.spark.serializer.KryoSerializer --conf spark.sql.hive.convertMetastoreParquet=false', '--enable-glue-datacatalog': 'true', '--datalake-formats': 'hudi'}


In [2]:


import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)



In [3]:
try:
    import os
    import sys


    import pyspark
    from pyspark import SparkConf, SparkContext
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, asc, desc
    from awsglue.utils import getResolvedOptions
    from awsglue.dynamicframe import DynamicFrame
    from awsglue.context import GlueContext

    #from faker import Faker

    print("All modules are loaded .....")

except Exception as e:
    print("Some modules are missing {} ".format(e))

All modules are loaded .....


In [4]:
database_name1 = "hudidb"
table_name = "hudi_table"
base_s3_path = "s3a://test-ramneek-2"
final_base_path = "{base_s3_path}/{table_name}".format(
    base_s3_path=base_s3_path, table_name=table_name
)

In [5]:
columns = ["ts","uuid","rider","driver","fare","city"]
data =[(1695159649087,"334e26e9-8355-45cc-97c6-c31daf0df330","rider-A","driver-K",19.10,"san_francisco"),
       (1695091554788,"e96c4396-3fad-413a-a942-4cb36106d721","rider-B","driver-L",27.70 ,"san_francisco"),
       (1695046462179,"9909a8b1-2d15-4d3d-8ec9-efc48c536a00","rider-C","driver-M",33.90 ,"san_francisco"),
       (1695516137016,"e3cf430c-889d-4015-bc98-59bdce1e530c","rider-C","driver-N",34.15,"sao_paulo")]
       

inserts = spark.createDataFrame(data).toDF(*columns)

In [6]:
inserts.show()

+-------------+--------------------+-------+--------+-----+-------------+
|           ts|                uuid|  rider|  driver| fare|         city|
+-------------+--------------------+-------+--------+-----+-------------+
|1695159649087|334e26e9-8355-45c...|rider-A|driver-K| 19.1|san_francisco|
|1695091554788|e96c4396-3fad-413...|rider-B|driver-L| 27.7|san_francisco|
|1695046462179|9909a8b1-2d15-4d3...|rider-C|driver-M| 33.9|san_francisco|
|1695516137016|e3cf430c-889d-401...|rider-C|driver-N|34.15|    sao_paulo|
+-------------+--------------------+-------+--------+-----+-------------+


In [7]:
hudi_options = {
    'hoodie.table.name': table_name,
}

In [8]:
inserts.write.format("hudi"). \
    options(**hudi_options). \
    mode("overwrite"). \
    save(final_base_path)

In [10]:
spark.read.format("org.apache.hudi").load(final_base_path).createOrReplaceTempView("hudi_snapshot")

spark.sql("select * from hudi_snapshot").printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- uuid: string (nullable = true)
 |-- rider: string (nullable = true)
 |-- driver: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- city: string (nullable = true)


In [11]:
columns1 = ["ts","uuid","rider","driver","fare","city","shift"]
data1 =[(1695159649033,"334e26e9-8355-45cc-97c6-c31daf0df330","rider-E","driver-K",19.10,"san_francisco", "A"),
       (1695091554732,"e96c4396-3fad-413a-a942-4cb36106d721","rider-F","driver-L",27.70 ,"san_francisco", "B")]

updatedDf = spark.createDataFrame(data1).toDF(*columns1)

updatedDf.write.format("hudi").options(**hudi_options).mode("append").save(final_base_path)



In [12]:
spark.read.format("org.apache.hudi").load(final_base_path).createOrReplaceTempView("hudi_snapshot")

spark.sql("select * from hudi_snapshot").printSchema()

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- ts: long (nullable = true)
 |-- uuid: string (nullable = true)
 |-- rider: string (nullable = true)
 |-- driver: string (nullable = true)
 |-- fare: double (nullable = true)
 |-- city: string (nullable = true)
 |-- shift: string (nullable = true)


In [13]:
spark.sql("select * from hudi_snapshot").show()

+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+-----+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|           ts|                uuid|  rider|  driver| fare|         city|shift|
+-------------------+--------------------+--------------------+----------------------+--------------------+-------------+--------------------+-------+--------+-----+-------------+-----+
|  20241007074939351|20241007074939351...|334e26e9-8355-45c...|                      |51cb6676-17b5-431...|1695159649033|334e26e9-8355-45c...|rider-E|driver-K| 19.1|san_francisco|    A|
|  20241007074939351|20241007074939351...|e96c4396-3fad-413...|                      |51cb6676-17b5-431...|1695091554732|e96c4396-3fad-413...|rider-F|driver-L| 27.7|san_francisco|    B|
|  20241007074222783|20241007074222783...|9909a8b1-2d15-4d3...|       